In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,514763
1,Huelva,Confirmados PDIA,73787
2,Huelva,Confirmados PDIA 14 días,5042
3,Huelva,Tasa PDIA 14 días,"979,4798771473475"
4,Huelva,Confirmados PDIA 7 días,1678
5,Huelva,Tasa PDIA 7 dias,"325,97525463174316"
6,Huelva,Total Confirmados,74046
7,Huelva,Curados,57535
8,Huelva,Fallecidos,445


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  73787.0


/tmp/ipykernel_23266/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  21512.0


/tmp/ipykernel_23266/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_23266/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_23266/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_23266/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 295 personas en los últimos 7 días 

Un positivo PCR cada 98 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,73787.0,5042.0,1678.0,514763.0,325.975255,979.479877,313.0
Huelva-Costa,42496.0,2472.0,802.0,290301.0,276.264980,851.529964,123.0
Sierra de Huelva-Andévalo Central,9992.0,1608.0,549.0,67400.0,814.540059,2385.756677,120.0
Huelva (capital),21512.0,1453.0,483.0,142538.0,338.857007,1019.377289,74.0
Condado-Campiña,20827.0,954.0,324.0,157062.0,206.287963,607.403446,69.0
Valverde del Camino,2230.0,381.0,134.0,12714.0,1053.956269,2996.696555,24.0
Aroche,410.0,89.0,39.0,3046.0,1280.367695,2921.864741,19.0
Lepe,3951.0,157.0,43.0,28293.0,151.981055,554.907574,14.0
Villalba del Alcor,608.0,43.0,30.0,3334.0,899.820036,1289.742052,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,292.0,77.0,39.0,1682.0,2318.668252,4577.883472,9.0
Cortegana,769.0,158.0,57.0,4635.0,1229.773463,3408.845739,11.0
Marines (Los),44.0,13.0,3.0,410.0,731.707317,3170.731707,0.0
Jabugo,322.0,71.0,23.0,2247.0,1023.587005,3159.768580,8.0
Nerva,726.0,156.0,64.0,5127.0,1248.293349,3042.715038,6.0
Valverde del Camino,2230.0,381.0,134.0,12714.0,1053.956269,2996.696555,24.0
Linares de la Sierra,24.0,8.0,1.0,268.0,373.134328,2985.074627,0.0
Valdelarco,14.0,7.0,1.0,236.0,423.728814,2966.101695,0.0
Aroche,410.0,89.0,39.0,3046.0,1280.367695,2921.864741,19.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,408.0,11.0,1.0,3478.0,28.752156,316.273721,0.0,0.090909
Alájar,71.0,9.0,1.0,814.0,122.850123,1105.651106,0.0,0.111111
Linares de la Sierra,24.0,8.0,1.0,268.0,373.134328,2985.074627,0.0,0.125000
Puerto Moral,39.0,7.0,1.0,281.0,355.871886,2491.103203,0.0,0.142857
Valdelarco,14.0,7.0,1.0,236.0,423.728814,2966.101695,0.0,0.142857
Niebla,613.0,34.0,6.0,4158.0,144.300144,817.700818,2.0,0.176471
Campillo (El),265.0,40.0,8.0,1996.0,400.801603,2004.008016,3.0,0.200000
Villanueva de los Castillejos,324.0,15.0,3.0,2849.0,105.300105,526.500527,0.0,0.200000
Almendro (El),89.0,5.0,1.0,835.0,119.760479,598.802395,0.0,0.200000
